In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pymysql
import pandas as pd
from bs4 import BeautifulSoup
import warnings # 경고창 무시
warnings.filterwarnings('ignore')

In [2]:
class Crolling:
    
    def __init__(self):
        self.conn = pymysql.connect(user= 'root',passwd  = '1234',host= "",port=3306,db='stock',charset = 'utf8')
        self.cur = self.conn.cursor()
        self.driver = webdriver.Chrome(ChromeDriverManager().install())
        self.driver.implicitly_wait(3)
        self.driver.maximize_window()
        
        
    def Stock_ID(self):
        for i in range(1, 21):
            url = f'https://finance.naver.com/sise/entryJongmok.naver?&page={i}'
            self.driver.get(url)
            self.driver.implicitly_wait(5)
            html = self.driver.page_source 
            soup = BeautifulSoup(html, 'html.parser')
            for j in range(3, 13):
                id = [i['href'].split("=")[1] for i in soup.select(f'body > div > table.type_1 > tbody > tr:nth-child({j}) > td.ctg > a')]
                name = [i.text for i in soup.select(f'body > div > table.type_1 > tbody > tr:nth-child({j}) > td.ctg > a')]
                sql = (id, name)
                self.cur.execute('INSERT IGNORE INTO Stock_ID (id, name) VALUES (%s ,%s)', sql)
                self.conn.commit()
            print(f'{i}/20 페이지 완료')
        self.driver.quit()
        
        
    def Stock_Price(self):
        self.cur.execute('SELECT id FROM Stock_ID;')
        stock_id = self.cur.fetchall()
        for id in stock_id:
            for i in range(1, 10):
                url = f'https://finance.naver.com/item/sise_day.naver?code={id[0]}&page={i}'
                self.driver.get(url)
                self.driver.implicitly_wait(5)
                html = self.driver.page_source 
                soup = BeautifulSoup(html, 'html.parser')
                for j in [3,4,5,6,7,11,12,13,14,15]:
                    date = soup.select_one(f'body > table.type2 > tbody > tr:nth-child({j}) > td:nth-child(1) > span').text
                    closing_price = soup.select_one(f'body > table.type2 > tbody > tr:nth-child({j}) > td:nth-child(2) > span').text.replace(',','')
                    market_price = soup.select_one(f'body > table.type2 > tbody > tr:nth-child({j}) > td:nth-child(4) > span').text.replace(',','')
                    high_price = soup.select_one(f'body > table.type2 > tbody > tr:nth-child({j}) > td:nth-child(5) > span').text.replace(',','')
                    low_price = soup.select_one(f'body > table.type2 > tbody > tr:nth-child({j}) > td:nth-child(6) > span').text.replace(',','')
                    sql = (id[0], date, closing_price, market_price, high_price, low_price)
                    self.cur.execute('INSERT IGNORE INTO Stock_Price3 (stock_id , date, closing_price, market_price, high_price, low_price) VALUES (%s ,%s ,%s ,%s ,%s ,%s)', sql)
                    self.conn.commit()
                    #break
                else:
                    continue
                if date == '2023.04.27':
                    break
        self.conn.close()
        self.driver.quit()
        
        
    def News(self):
        self.cur.execute('SELECT id FROM Stock_ID;')
        stock_id = self.cur.fetchall()
        for idx,id in enumerate(stock_id):
            for i in range(1, 500):
                url = f'https://finance.naver.com/item/news_news.naver?code={id[0]}&page={i}&sm=entity_id.basic&clusterId='
                self.driver.get(url)
                self.driver.implicitly_wait(5)
                html = self.driver.page_source 
                soup = BeautifulSoup(html, 'html.parser')
                #date = [i.text for i in self.driver.find_elements_by_class_name("date")]
                date = [i.text for i in self.driver.find_elements(By.CLASS_NAME,"date")]
                for j in soup.select('body > div > table.type5 > tbody'):
                    data = j.find_all("a")
                    data = [i["href"] for i in data if i["href"] != "#"]
                    for da,de in zip(data,date):
                        if de == '':
                            continue
                        if de < '2023.05.01':
                            break
                        try:
                            url = "https://finance.naver.com" + da
                            self.driver.get(url)
                            news_text_elements = self.driver.find_elements(By.ID, 'news_read')
                            if len(news_text_elements) > 0:
                                news_text = news_text_elements[0].text
                                self.cur.execute('INSERT IGNORE INTO Stock_News (stock_id, text, date) VALUES (%s ,%s, %s)', (id[0], news_text, de))
                                self.conn.commit()
                                #print(f"Inserted stock_id={id[0]}, date={de}")
                            else:
                                print("해당 요소를 찾지 못했습니다.")
                        except Exception as e:
                            print(f"에러가 발생했습니다 : {e}")
                            continue
                if de == '':
                    continue
                if de < '2023.05.01':
                    break
            print(f'{idx+1}/{len(stock_id)}')
        self.conn.close()
        self.driver.quit()
    

Crolling().Stock_Price()

In [3]:
# DB에 저장된 레코드, CSV파일로 export하기

import csv
import pymysql

def export_to_csv():
    db_host = 'localhost'
    db_user = 'root'
    db_password = '1234'
    db_name = 'stock'

    # DB 연결
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name)
    cur = conn.cursor()

    # 쿼리 실행
    cur.execute('SELECT * FROM Stock_Price3;')
    data = cur.fetchall()

    # CSV 파일로 내보내기
    with open('stock_price.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['stock_id', 'date', 'closing_price', 'market_price', 'high_price', 'low_price'])
        csv_writer.writerows(data)

    conn.close()

export_to_csv()

## 중복 제거된 뉴스 기사 csv로 내보내기

In [1]:
import csv
import pymysql

def export_to_csv():
    db_host = 'localhost'
    db_user = 'root'
    db_password = '1234'
    db_name = 'stock'

    # DB 연결
    conn = pymysql.connect(host=db_host, user=db_user, password=db_password, db=db_name)
    cur = conn.cursor()

    # 쿼리 실행
    cur.execute('SELECT DISTINCT stock_id, text, date FROM stock_news;')
    data = cur.fetchall()

    # CSV 파일로 내보내기
    with open('stock_distinct_news.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['stock_id', 'text', 'date'])
        csv_writer.writerows(data)

    conn.close()

export_to_csv()